# Basics on Markov chain Monte Carlo and physics applications

The aim of the Markov chain Monte Carlo (MCMC) is to produce random variables obeying any probability distributions as you like. There are three main tools for MCMC and I will introduce them one by one.

* Metropolis-Hastings algorithm
* Gibbs sampler
* Hybrid Monte Carlo

## Metropolis-(Hastings) algorithm

The Metropolis-Hastings (MH) algorithm is the most important and most popular tool in MCMC, Bayesian statistics, and physics simulations. It is universal and applicable to any kinds of distributions. The other methods usually have some restrictions, so if you do not know how to use another one like a Gibbs sampler, you should choose the MH method.

### Classical Monte Carlo simulation for Ising model

As a typical example, I will present a simple Julia implimentation of the Monte Carlo simulation of the 2D Ising model on the square lattice. The Metropolis rule is simply given by the following function.

In [1]:
function Metropolis(β::Float64,ΔE::Float64)
    return (ΔE <= 0.) || (exp(-β * ΔE) > rand())
end

Metropolis (generic function with 1 method)

This function will tell us whether we will accept a new state or not according to its energy gain $\Delta E$. It is clear that acceptance/rejection will be determined probabilistically and typically the MH alrorithm has an acceptance rate of only 20-40% (c.f. in the Ising model it is much higher). I will later show you algorithms with an almost 100% acceptance rate, such as a Gibbs sampler or a hybrid Monte Carlo.

### Storing spin configurations
The most memory-effcient way to store a spin configuration is using bitwise operations. Usually we need a very low-level code for such bitwise operations (see e.g. "Hacker's Delight" by H. S. Warren), but Julia provides a nice platform called BitArray. We regard a bit 1 as spin-up ($\sigma =+1$) and 0 as spin-down ($\sigma =-1$). For 1D, we can just store an array of spins as BitArray.

In [2]:
L = 8
spin1D = trues(L)

8-element BitArray{1}:
 true
 true
 true
 true
 true
 true
 true
 true

In order to store a 2D spin configuration, Array of BitArray is more useful than 2D BitArray (you will understand later).

In [3]:
spin2D = rand(Bool, L, L)
spin2Drow = [BitVector(spin2D[n, :]) for n in 1:L]

8-element Array{BitArray{1},1}:
 Bool[false, false, true, true, true, true, true, true]   
 Bool[true, false, false, true, true, true, false, true]  
 Bool[true, false, true, true, false, true, true, false]  
 Bool[false, false, false, true, true, true, false, false]
 Bool[true, true, false, true, false, true, true, false]  
 Bool[false, false, true, false, true, true, true, false] 
 Bool[true, false, false, false, true, true, true, false] 
 Bool[true, false, false, false, true, false, true, true] 

For the futer purpose, it is useful later to store the transposed version:

In [4]:
spin2Dcolumn = [BitVector(spin2D[:, n]) for n in 1:L]

8-element Array{BitArray{1},1}:
 Bool[false, true, true, false, true, false, true, true]    
 Bool[false, false, false, false, true, false, false, false]
 Bool[true, false, true, false, false, true, false, false]  
 Bool[true, true, true, true, true, false, false, false]    
 Bool[true, true, false, true, false, true, true, true]     
 Bool[true, true, true, true, true, true, true, false]      
 Bool[true, false, true, false, true, true, true, true]     
 Bool[true, true, false, false, false, false, false, true]  

### Updating
Assuming a periodic boundary condition, the nearest-neighbor sites can be obtained by bitwise shift (rotation) operations. It is appropriate for the small $L$ but for a larger size it has a large overhead.

In [5]:
J = -1.0 # Ferro
function energygain(x, y, row, column)
    left = xor(ror(row[y],1)[x], row[y][x])
    right = xor(rol(row[y],1)[x], row[y][x])
    top = xor(ror(column[x],1)[y], column[x][y])
    bottom = xor(rol(column[x],1)[y], column[x][y])
    return -2. * J * sum(1 - 2 * [left,right,top,bottom])
end
energygain(3, 1, spin2Drow, spin2Dcolumn)

-4.0

If we choose the flipping spin sequentially (sequential sweep), it may destroy a detailed balance microscopically. Usually it is better to choose a candidate spin using a random variable. Note that functions can be used as arguments in Juila.

In [6]:
function step(β, method, row, column)
    x = rand(1:L)
    y = rand(1:L)
    ΔE = energygain(x, y, row, column)
    flip = method(β, ΔE)
    row[y][x] = xor(row[y][x], flip)
    column[x][y] = xor(column[x][y], flip)
    return(x, y, flip)
end
step(0.1, Metropolis, spin2Drow, spin2Dcolumn)

(6, 2, true)

Julia is using "call by reference" for Array, so spin2Drow will be directely updated.

In [7]:
spin2Drow

8-element Array{BitArray{1},1}:
 Bool[false, false, true, true, true, true, true, true]   
 Bool[true, false, false, true, true, false, false, true] 
 Bool[true, false, true, true, false, true, true, false]  
 Bool[false, false, false, true, true, true, false, false]
 Bool[true, true, false, true, false, true, true, false]  
 Bool[false, false, true, false, true, true, true, false] 
 Bool[true, false, false, false, true, true, true, false] 
 Bool[true, false, false, false, true, false, true, true] 

### Simulations
Let's warm up!

In [8]:
function warmup(β)
    N = 10000000
    for i in 1:N
        step(β, Metropolis, spin2Drow, spin2Dcolumn)
    end
end
warmup(0.1)
spin2Drow

8-element Array{BitArray{1},1}:
 Bool[true, false, true, true, false, false, false, true] 
 Bool[false, false, true, true, true, false, true, false] 
 Bool[true, true, true, false, true, true, false, false]  
 Bool[true, false, false, true, true, true, true, true]   
 Bool[false, true, false, false, true, false, true, true] 
 Bool[true, true, false, false, false, false, true, false]
 Bool[true, true, true, true, true, false, false, false]  
 Bool[true, true, true, true, true, false, false, true]   

At low temperature, the spins will be ordered to the ferromagnetic state.

In [9]:
warmup(10.0)
spin2Drow

8-element Array{BitArray{1},1}:
 Bool[true, true, true, true, true, true, true, true]
 Bool[true, true, true, true, true, true, true, true]
 Bool[true, true, true, true, true, true, true, true]
 Bool[true, true, true, true, true, true, true, true]
 Bool[true, true, true, true, true, true, true, true]
 Bool[true, true, true, true, true, true, true, true]
 Bool[true, true, true, true, true, true, true, true]
 Bool[true, true, true, true, true, true, true, true]

## Gibbs sampler (heat-bath method)
A Gibbs sampler is the most powerful tool in MCMC if you are able to generate a "conditional probability distribution." Only in this case, we can achieve a 100% acceptance rate for the continuous variable distributions, i.e. the acceptance/rejection process is deterministic. For some reason a Gibbs sampler is called heat-bath method in physics, but you should learn it in this generalized form.

### Example

Assuming we want to sample from a distribution defined by a probablity
$$
p(x,y) \propto e^\frac{-a x^2+2b xy - cy^2}{2},
$$
we can explicitly derive a conditional distribution as a Gaussian distribution for each variable.
$$
p(x|y) \propto e^{-\frac{a}{2} \left(x - \frac{b}{a}y\right)^2}, \\
p(y|x) \propto e^{-\frac{c}{2} \left(y - \frac{b}{c}x\right)^2}.
$$

By updating $x$ and $y$ alternately from these two conditional distributions, we can produce the original joint distribution because it is the invariant distribution of these two sampling processes. Note that the detailed balance is microscopically broken, but the algorithm is balanced as a total. If you wish to resotre it, it is better to choose $x$ or $y$ randomly.

In [10]:
using Distributions
function Gibbs_sampler(a::Float64, b::Float64, c::Float64, x, y)
    x = rand(Normal(b / a * y, 1 / sqrt(a)))
    y = rand(Normal(b / c * x, 1 / sqrt(c)))
    return x, y
end
Gibbs_sampler(1.0, 0.8, 1.0, 0.0, 0.0)

(-0.9119302107437514, -1.3611504410685207)

Julia includes the library for various distributions, so the implementation should be simple. After the convergence it will give us a correct distribution.

In [11]:
function sampling(a, b, c)
    N = 1000000
    x = 0.0
    y = 0.0
    for i in 1:N
        x, y = Gibbs_sampler(a, b, c, x, y)
    end
    return x, y
end
sampling(1.0, 0.8, 1.0)

(-1.0726735979266135, -1.1551881177101888)

### For Ising model

For the Ising model, the Gibbs sampler will reduce to just a so-called heat-bath method probably because the unupdated spins can be regarded as a heat bath and the transition probablities obey the partial "canonical distribution" for the updated spin. The implementation is simple and can be done just by replacing the function Metropolis to the following one:

In [12]:
function heatbath(β::Float64,ΔE::Float64)
    return (1. / (exp(+β * ΔE) + 1.)) > rand()
end
step(0.1, heatbath, spin2Drow, spin2Dcolumn)

(3, 8, false)

Note that the acceptance rate is apparently not 100% due to the finite number of the possible transitions.

In [13]:
function heatup(β)
    N = 10000000
    for i in 1:N
        step(β, heatbath, spin2Drow, spin2Dcolumn)
    end
end
heatup(0.1)
spin2Drow

8-element Array{BitArray{1},1}:
 Bool[true, true, true, false, false, false, false, false] 
 Bool[false, true, false, true, false, false, false, false]
 Bool[false, true, true, true, false, false, false, false] 
 Bool[true, false, false, true, false, true, false, false] 
 Bool[false, false, false, false, true, true, false, false]
 Bool[false, false, true, true, true, true, false, true]   
 Bool[false, true, false, true, true, true, true, true]    
 Bool[true, true, false, true, false, false, false, true]  

In [17]:
heatup(10.0)
spin2Drow

8-element Array{BitArray{1},1}:
 Bool[false, false, false, false, false, false, false, false]
 Bool[false, false, false, false, false, false, false, false]
 Bool[false, false, false, false, false, false, false, false]
 Bool[false, false, false, false, false, false, false, false]
 Bool[false, false, false, false, false, false, false, false]
 Bool[false, false, false, false, false, false, false, false]
 Bool[false, false, false, false, false, false, false, false]
 Bool[false, false, false, false, false, false, false, false]

### Blocked Gibbs sampler

From now on, I will introduce some variants of the original Gibbs sampler.

### Collapsed Gibbs sampler

## Hybrid Monte Carlo

Condensed matter physicists do not usually use this method because they are mostly using discrete variables, but this method is the most efficient method with an almost 100% acceptance rate "for continuous variables" because it is using a gradient of the probability distribution. A hybrid Monte Carlo or Hamiltonian Monte Carlo (HMC) would be the best solution for the distribution with a known gradient and it is much more efficient than the pure MH method because of the highest acceptance rate.

The essence of HMC can be understood as some hybrid of the MH method and the Gibbs sampler (or deterministic molecular dynamics), and HMC generates an extended probability distribution including an auxiliary field called "momentum," where the conditional probability distribution without this momentum becomes the disired one. In some quantum Monte Carlo (QMC) simulations, it is difficult to calculate the gradient explicitly, and then we have to introduce another auxiliary field (called "pseudofermion" although it is actually bosonic) to estimate the gradient probabilistically "as an avarage."

### How it works

Instead of generating the original distribution $\exp(-V(\chi))$ (e.g. $V(\chi) = \beta E(\chi)$ for the continuous variables $\chi$), we would like to generate a joint distribution of $\chi$ and $\pi$ (if $\chi$ is a vector, $\pi$ should have the same dimention):
$$p(\chi,\pi)=\exp(-\pi^2-V(\chi)).$$
If we simply discard the auxiliary variables $\pi$, the resulting conditional distribution simply becomes the original one.
$$p(\chi | \pi)=\exp(-V(\chi)).$$
Thus it is enough to generate the extended distribution with doubled variables.

If we define a classical Hamiltonian as $\mathcal{H}=\pi^2+V(\chi)$, the problem results in the generation of the distribution of the classical probabilistic system with $p=\exp(-\mathcal{H})$. Note that we simply set the mass $m=1/2$, but we can generalize it to any mass matrix.

### Algorithm

The updating process goes similarly to the Gibbs sampler. First, we update $\pi$ from
$$p(\pi | \chi)=\exp(-\pi^2).$$
This is just a normal distribution, so Box–Muller's method simply works for this step.

We use a deterministic molecular dynamics to update $\chi$. Since the equation of motion conserves the total energy, the distribution is not changed by this motion from $t=0$ to any time $t=T$ when we update both $\chi$ and $\pi$ simultaneously according to the canonical equation. Typically, we move $(\chi,\pi)$ for $T=1$ according to the randomly selected momentum $\pi$ in order to move $\chi$ for a long enough distance (to avoid the self-correlation). The detailed balance completely holds for this deterministic updating *as long as the equation of motion is solved exactly without any numerical error.* Under this (numerically impossible) assumption, we can achieve 100% acceptance rate for this HMC algorithm.

### Leap-flogging

In the ideal case with a negligible numerical error, HMC can achieve almost 100% acceptance rate and in this sense it is very efficient, but this is impossible in a real computer using floating-point numbers. Thus, we use the following integrator for the equation of motion to keep its detailed balance:

1. $\chi(t+\Delta t/2)  = \chi(t) + \pi(t) \Delta t.$
1. $\pi(t+ \Delta t) = \pi(t) - \nabla V(\chi+\Delta t/2) \Delta t.$
1. $\chi(t+\Delta t)  = \chi(t+\Delta t/2) + \pi(t+ \Delta t) \Delta t.$

This algorithm is apparently reversible, i.e. everything goes back when we replace $\pi \to -\pi$. This fact will not be affected by the numerical error because the unupdated variable is conserved for each step (see e.g. the lifting algorithm for the wavelet transformation). Thus, the detailed balance (i.e. reversibility of the process) is still kept even with a numerical error for this leap-flogging approximation considering the fact that the normal distribution produces the same amount of $\pi$ and $-\pi$.

However, if $\Delta t$ is not small enough, the energy $\mathcal{H}$ is no longer conserved due to the numerical error and we need an acceptance/rejection process to keep its distribution. This is simply realized by the MH algorithm for the accumulated error for the energy $\Delta \mathcal{H} = \mathcal{H}_\textrm{new} - \mathcal{H}_\textrm{old}$. Note that we are dealing with a probability density, not a probability itself, so the measure $d\chi d\pi$ of $p(\chi,\pi)d\chi d\pi$ also has to be conserved. The leap-flogging has no problem on this point because it clearly conserves the symplectic form $d\chi d\pi$ regardless of the error (take the exact differential of each step).

If we begin from the MH algorithm (other way around), HMC can be regarded as an improved version of the random walk + MH algorithm. If we just do a random walk (with a momentum $\pi$) and check it by MH, there is no reason to think that the energy difference $\Delta \mathcal{H} = \mathcal{H}_\textrm{new} - \mathcal{H}_\textrm{old}$ becomes small. However, if we use a molecular dynamics instead of the random walk, the origin of $\Delta \mathcal{H}$ is totally the numerical error and we can expect that it becomes small if the calculation is accurate enough. HMC is one simple way to increase the acceptance rate of the MH algorithm to almost 100%.

### Implementation

In [14]:
function leapflogging(χ, π, Δt::Float64, N, ∇V)
    χ += π * Δt
    for i in 1:(N-1)
        π -= ∇V(χ) * Δt
        χ += π * 2Δt
    end
    π -= ∇V(χ) * Δt
    χ += π * Δt
    return χ, π
end

leapflogging (generic function with 1 method)

In [15]:
function HMC(χ::Vector{Float64}, V::Function, ∇V::Function)
    Nstep = 1000
    Nintegrate = 10
    Δt = 1. / Nintegrate
    π = similar(χ)
    for i in 1:Nstep
        π = rand(Normal(0, 1 / sqrt(2)), length(π))
        Hold = dot(π, π) + V(χ)
        χold = χ
        πold = π
        χ, π = leapflogging(χ, π, Δt, Nintegrate, ∇V)
        Hnew = dot(π, π) + V(χ)
        println((Hold, Hnew))
        if Metropolis(1., Hnew - Hold)
            println("Accepted!")
        else
            χ = χold
            π = πold
        end
    end
    return χ
end
v = x -> (x[1] ^ 2 - 2 * 0.8x[1] * x[2] + x[2]^2) / 2.
dv = x -> [x[1] - 0.8x[2], x[2] - 0.8x[1]]
HMC([0., 0.], v, dv)

(0.14608411878330385, 0.14494142525063927)
Accepted!
(0.14013143493608202, 0.141131874164843)
Accepted!
(0.05004403552432202, 0.05011314214099313)
Accepted!
(0.05815320767665064, 0.05791847225517776)
Accepted!
(3.571582798156838, 3.546186465999905)
Accepted!
(4.691925248598196, 4.7172432901143395)
Accepted!
(0.7848236823317497, 0.781607234786742)
Accepted!
(1.3730608009533283, 1.3750799841083876)
Accepted!
(1.184289687623616, 1.1801524856796306)
Accepted!
(0.6482767141923205, 0.6528193494632691)
Accepted!
(0.4502108579057595, 0.45071999171225774)
Accepted!
(0.8943041535668128, 0.8879504812409044)
Accepted!
(1.8325467760184884, 1.8384532703074927)
Accepted!
(1.4744317275616077, 1.4663779658694878)
Accepted!
(2.0133685789284654, 2.0224221507758595)
Accepted!
(0.13186662158344198, 0.131428897213673)
Accepted!
(0.3055776887369239, 0.30606983266428595)
Accepted!
(1.3683435319986899, 1.366647866523192)
Accepted!
(3.91372230590079, 3.899879360908063)
Accepted!
(4.084552140072359, 4.0927057973

(0.6385452952645381, 0.6413188227899942)
Accepted!
(2.8450609836333896, 2.844027932000747)
Accepted!
(2.1770638529523, 2.178279590229287)
Accepted!
(2.152751633448871, 2.1504812152150206)
Accepted!
(2.4566594662129946, 2.4529995247526646)
Accepted!
(3.3305866273898204, 3.3231560634507744)
Accepted!
(3.4086041978153636, 3.419963612528685)
Accepted!
(2.0911512908093983, 2.0936163085491817)
Accepted!
(4.234780204078016, 4.235744157778087)
Accepted!
(0.5788225728729487, 0.5788491705869439)
Accepted!
(1.3085229557535887, 1.302518940280665)
Accepted!
(3.0049280222828774, 2.9937058649387565)
Accepted!
(2.3355258602360807, 2.352300164761064)
Accepted!
(0.3619372453045673, 0.36261493134789613)
Accepted!
(1.9652958448082793, 1.9641959377961102)
Accepted!
(1.4623827985362032, 1.4630920149946247)
Accepted!
(3.1806460529950207, 3.176909874597656)
Accepted!
(2.755642378129129, 2.755084342551709)
Accepted!
(1.1022860868640405, 1.1064966075640448)
Accepted!
(0.6721376692655976, 0.6716695881841288)
Acc

(2.3154689908963224, 2.3071691155486524)
Accepted!
(2.200621226741323, 2.210128106158642)
Accepted!
(1.4137802062167772, 1.4128549888525916)
Accepted!
(2.4290117666656883, 2.426714061497437)
Accepted!
(1.5904825340636224, 1.593299010466945)
Accepted!
(0.7443407351022582, 0.7449389965313671)
Accepted!
(4.352059720629074, 4.3362770441908856)
Accepted!
(2.1037195371345603, 2.12011991519137)
Accepted!
(0.19873725260621275, 0.19816175355910304)
Accepted!
(0.7616615035190382, 0.759497372310642)
Accepted!
(0.6201772675230379, 0.6216002896314208)
Accepted!
(0.41507544348548725, 0.41439889166694177)
Accepted!
(3.751637254791091, 3.727652368677165)
Accepted!
(3.9892680293266536, 4.01351242615152)
Accepted!
(1.5652724621268113, 1.5549807209805753)
Accepted!
(4.530905102236174, 4.533270046138901)
Accepted!
(1.1456155881036323, 1.1542228704464108)
Accepted!
(1.7350551968854635, 1.730567042644772)
Accepted!
(2.4697799098687847, 2.4621501562387875)
Accepted!
(2.2294775477033237, 2.242121946071845)
Ac

(1.043511048111594, 1.0448193337209803)
Accepted!
(1.1536311116164026, 1.150875972221641)
Accepted!
(1.3375875960289902, 1.3382441840295405)
Accepted!
(0.8263057293442149, 0.8262296103049017)
Accepted!
(1.4216401108838348, 1.4160057533696677)
Accepted!
(2.0211886942044104, 2.028217099703789)
Accepted!
(1.1927927182163987, 1.1937321330318538)
Accepted!
(1.330623853623286, 1.330444772020361)
Accepted!
(1.3577835732136587, 1.3564519032200628)
Accepted!
(0.452806504002348, 0.45489724436901985)
Accepted!
(1.5503812576944924, 1.5495014599278008)
Accepted!
(1.9485341573014954, 1.94248373316094)
Accepted!
(1.362395486474646, 1.3680403025492665)
Accepted!
(0.8179041683041437, 0.8164553247063682)
Accepted!
(0.8552232055827035, 0.8574981240043204)
Accepted!
(0.6880762368085189, 0.6868730477517548)
Accepted!
(0.9978024900378234, 0.9960320592026748)
Accepted!
(0.6722346569844984, 0.6724184679531027)
Accepted!
(2.7666375859274264, 2.7693800349676874)
Accepted!
(1.1417174753930115, 1.1381238054709668

(1.5402377227323765, 1.546877124692276)
Accepted!
(1.7585131753226593, 1.7538867705554209)
Accepted!
(1.0929695689985999, 1.0923920541469174)
Accepted!
(1.9421384948291873, 1.9427868753510509)
Accepted!
(1.558371120217131, 1.5519149777318975)
Accepted!
(1.7237308005473255, 1.7351426095669003)
Accepted!
(2.3012298196905103, 2.2966185033945767)
Accepted!
(4.419474385320787, 4.401983601734318)
Accepted!
(5.45681009241839, 5.466724099792685)
Accepted!
(2.7795260938755106, 2.7916684629550534)
Accepted!
(0.7830683798070105, 0.7828871490615269)
Accepted!
(0.598394811319794, 0.5962695962908552)
Accepted!
(0.7333815143671446, 0.7343033129400225)
Accepted!
(1.0346451785458857, 1.0308237399286306)
Accepted!
(1.3525422399007687, 1.348666632173411)
Accepted!
(1.6634825808889402, 1.6691462727457251)
Accepted!
(1.2216206144603878, 1.2241533400714337)
Accepted!
(0.9036317000352568, 0.9037318271353065)
Accepted!
(1.6906277025710965, 1.6831269922466205)
Accepted!
(1.6537911759671169, 1.6606918854498025)

2-element Array{Float64,1}:
 -2.07003
 -2.78909

### Automatic Differentiation

Now in the field of machine learning or Bayesian statistics, the "autograd" i.e. automatic differentiation becomes one of the most powerful tools. Physicists still like to do differentiations by themselves, but a functional language like Julia can differentiate the function directly, so we no longer need any paper/pen.

In [16]:
using ForwardDiff
HMC([0., 0.], v, x -> ForwardDiff.gradient(v, x))

(0.30927930596098746, 0.3073463323300858)
Accepted!
(1.2243723803592683, 1.2260499595966903)
Accepted!
(0.44293673748737195, 0.44233305608841134)
Accepted!
(1.3441694243788889, 1.3420073744232268)
Accepted!
(0.6169947795980627, 0.6172335681653112)
Accepted!
(1.1631273864199392, 1.1649341075331408)
Accepted!
(0.6852438888727497, 0.6822292553534594)
Accepted!
(2.021784642914483, 2.0246332628623467)
Accepted!
(2.6881461072744632, 2.676690749742459)
Accepted!
(3.1597623502335703, 3.1515598496577235)
Accepted!
(3.13915264727829, 3.1545159547687627)
Accepted!
(1.3813576063388622, 1.383740391619534)
Accepted!
(0.40826623264796547, 0.4111907368863265)
Accepted!
(2.0561042247476498, 2.055368738083925)
Accepted!
(1.6691869809018995, 1.6629272823396934)
Accepted!
(2.9676558577548215, 2.9723774446972744)
Accepted!
(1.97389104001751, 1.9748795710176346)
Accepted!
(3.642756657607407, 3.6338808510907366)
Accepted!
(3.712081818052575, 3.7211584797923787)
Accepted!
(1.3333802379664477, 1.33282916003461

(1.40839027860644, 1.4108125579670814)
Accepted!
(2.785065216993504, 2.7706413347861636)
Accepted!
(2.3928063348540496, 2.407200831059888)
Accepted!
(0.9943792652696547, 0.988619617315409)
Accepted!
(2.490517418622444, 2.4948963606905803)
Accepted!
(1.5133265510961451, 1.5114880246288382)
Accepted!
(2.315967837172087, 2.317513776850128)
Accepted!
(3.8234140470127445, 3.8120519491234184)
Accepted!
(5.109595746922229, 5.106260281237132)
Accepted!
(6.1633772321845735, 6.175217947296636)
Accepted!
(5.228329745492864, 5.229620590253121)
Accepted!
(2.108306175994412, 2.1104983501862478)
Accepted!
(2.011568950540867, 2.0123773768801683)
Accepted!
(1.2336916253272774, 1.2340309852331033)
Accepted!
(0.7225879367263025, 0.7210810688154221)
Accepted!
(0.7483479906341166, 0.7499632088325414)
Accepted!
(1.4391826779318593, 1.43953130496449)
Accepted!
(0.2725904531564646, 0.27178007049173525)
Accepted!
(3.5998891021283743, 3.58734708940512)
Accepted!
(2.5594666547872897, 2.560127787216074)
Accepted!

(2.501670873169863, 2.5056629752553112)
Accepted!
(2.936356657064397, 2.9298169914557484)
Accepted!
(2.0259825036996624, 2.0312154985216555)
Accepted!
(1.8241019709356363, 1.82418378354923)
Accepted!
(2.066334110558903, 2.0636012758374944)
Accepted!
(2.729004659021488, 2.7316176118584914)
Accepted!
(1.9396951869246843, 1.9405970855622503)
Accepted!
(1.7091897995794958, 1.7112935612177456)
Accepted!
(0.2914872311860954, 0.29123469188971596)
Accepted!
(0.8095828793718789, 0.8063347787681581)
Accepted!
(0.8591140939886245, 0.8583956580230689)
Accepted!
(2.0584584581344085, 2.057172392949953)
Accepted!
(0.9107246400441121, 0.916176264015644)
Accepted!
(0.880673410924822, 0.8757625781477824)
Accepted!
(1.1040309739444676, 1.1067805071382417)
Accepted!
(0.9009594664778497, 0.9015004777847047)
Accepted!
(1.6685491472128609, 1.670417332125687)
Accepted!
(0.8120710516628019, 0.8071869472068754)
Accepted!
(1.8030858618657768, 1.8047179532677893)
Accepted!
(4.320371959489913, 4.292446599820905)
A

(0.8674272904537594, 0.8669166464826117)
Accepted!
(1.9005812730904739, 1.8977509041294163)
Accepted!
(4.241975203933983, 4.237906506193538)
Accepted!
(4.3877398032661485, 4.391543925064896)
Accepted!
(4.128883647685273, 4.1182957752059135)
Accepted!
(4.432603300432589, 4.4461809418668965)
Accepted!
(1.5732980058158397, 1.5683760354519949)
Accepted!
(2.4943470406500574, 2.4935705125662633)
Accepted!
(1.349610807727578, 1.3491795472454053)
Accepted!
(2.649917760576431, 2.656671225706796)
Accepted!
(2.1364299507082904, 2.1221196950852934)
Accepted!
(4.14401070514033, 4.152144611918051)
Accepted!
(0.8335351505826814, 0.8380449216897552)
Accepted!
(0.7034354324781743, 0.7024849681696351)
Accepted!
(0.7863174374850586, 0.7857008740448477)
Accepted!
(1.6987069900023208, 1.7001977681229699)
Accepted!
(1.7375418811606171, 1.7385169898059358)
Accepted!
(0.23484659513132228, 0.23541663391158646)
Accepted!
(0.6777865303572859, 0.678114092266195)
Accepted!
(0.7324753346333499, 0.7312999146713911)


(0.8498641106372298, 0.8495668940335659)
Accepted!
(1.2052969814072465, 1.2052151766396475)
Accepted!
(1.4685453102825448, 1.4655537833678003)
Accepted!
(1.5442483851201771, 1.5470201565569934)
Accepted!
(0.9837145761020467, 0.9854197281027555)
Accepted!
(0.8698327096555019, 0.8693322920370885)
Accepted!
(2.638865797762909, 2.626877160564524)
Accepted!
(2.426766746365928, 2.429571129428191)
Accepted!
(3.2516901904395636, 3.2466346758117837)
Accepted!
(3.803883708332854, 3.801683643670055)
Accepted!
(2.580976012179645, 2.5958753876386305)
Accepted!
(1.1113368965860018, 1.112558234721999)
Accepted!
(1.6485184147405056, 1.6432344645309698)
Accepted!
(1.9911283704337124, 1.9935498562110405)
Accepted!
(2.4534205785485823, 2.4544797263991835)
Accepted!
(2.5332546456296905, 2.5299428611081565)
Accepted!
(2.0123431800706766, 2.0169757466519975)
Accepted!
(1.5596139986287114, 1.5594111543171802)
Accepted!
(1.1799337851507214, 1.1809443895505967)
Accepted!
(1.3878148604039033, 1.3858551954763147

2-element Array{Float64,1}:
 3.30279
 2.3753 

1. https://github.com/JuliaDiff/ForwardDiff.jl